In [1]:
from yolov3 import make_new_seq_yolov3_model
from utils.utils import WeightReader
from utils.utils import preprocess_input
from utils.utils import decode_netout
from utils.utils import correct_yolo_boxes
from utils.utils import do_nms
from utils.bbox import draw_boxes
from utils.utils import get_yolo_boxes
from keras.models import Sequential
from keras.layers import  Dense

import copy
import time
import keras.backend as K
import numpy as np
import cv2
import os

Using TensorFlow backend.


In [2]:
net_h, net_w = 416, 416
obj_thresh, nms_thresh = 0.5, 0.45
anchors = [10, 13, 16, 30, 33, 23, 30, 61, 62, 45, 59, 119, 116, 90, 156, 198, 373, 326]
labels = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", \
          "boat", "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", \
          "bird", "cat", "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", \
          "backpack", "umbrella", "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", \
          "sports ball", "kite", "baseball bat", "baseball glove", "skateboard", "surfboard", \
          "tennis racket", "bottle", "wine glass", "cup", "fork", "knife", "spoon", "bowl", "banana", \
          "apple", "sandwich", "orange", "broccoli", "carrot", "hot dog", "pizza", "donut", "cake", \
          "chair", "sofa", "pottedplant", "bed", "diningtable", "toilet", "tvmonitor", "laptop", "mouse", \
          "remote", "keyboard", "cell phone", "microwave", "oven", "toaster", "sink", "refrigerator", \
          "book", "clock", "vase", "scissors", "teddy bear", "hair drier", "toothbrush"]

imgs_folder_path = 'Z:/dataset/KNU-Campus Dataset/images/20180312_172240/'
img_name = '20180312_172240_'
infer_model,seq_infer_model = make_new_seq_yolov3_model(input_shape=(416,416,3))

if not (os.path.exists('Weights/yolov3.h5')):
    weight_reader = WeightReader('yolov3.weights')
    weight_reader.load_weights(infer_model)
    infer_model.save_weights('Weights/yolov3.h5')
    seq_infer_model.load_weights('Weights/yolov3.h5', by_name=True)
else :
    infer_model.load_weights('Weights/yolov3.h5',by_name=True)
    seq_infer_model.load_weights('Weights/yolov3.h5', by_name=True)
    
output_layer_weights = [infer_model.get_layer(name='conv_81').get_weights()[0],
                        infer_model.get_layer(name='conv_93').get_weights()[0],
                        infer_model.get_layer(name='conv_105').get_weights()[0]]

In [9]:
def get_grad(yolo_num,row,col,out_idx):
    return output_layer_weights[yolo_num][0,0,:,out_idx]

In [ ]:
prev_feature=[]
for img_num in range(900):
        
    img_num2str=str(img_num)
    while(len(img_num2str)<4):
        img_num2str='0'+img_num2str

    img = cv2.imread(imgs_folder_path+img_name+img_num2str+'.jpg')
    image_h, image_w, _ = img.shape
    
    process_image = preprocess_input(img, net_h, net_w)
    
    if img_num == 0:
        yolos = infer_model.predict(process_image)
        prev_feature = copy.deepcopy(yolos[:3])
        yolos = yolos[:3]
    else:
        start_time = time.time()
        yolos = infer_model.predict(process_image)
        update_layer = yolos[3:]
        
        l_r = 0.5
        
        flag=[0]*3
        gradients=[np.zeros(shape=(13,13,1024)),np.zeros(shape=(26,26,512)),np.zeros(shape=(52,52,256))]
        count=[np.zeros(shape=(13,13,1024)),np.zeros(shape=(26,26,512)),np.zeros(shape=(52,52,256))]
        
        for i in range(len(rows)):
            row = rows[i]
            col = cols[i]
            b = bs[i]
            class_num = class_nums[i]
            yolo_num = yolo_nums[i]
            if (flag[yolo_num] == 0):
                gradients[yolo_num][row,col,:] = get_grad(yolo_num = yolo_num, row = row, col = col, out_idx = 85*b +4) 
                gradients[yolo_num][row,col,:] += get_grad(yolo_num = yolo_num, row = row, col = col, out_idx = 85*b +5+ class_num)
            else:
                gradients[yolo_num][row,col,:] += get_grad(yolo_num = yolo_num, row = row, col = col, out_idx = 85*b +4)
                gradients[yolo_num][row,col,:] += get_grad(yolo_num = yolo_num, row = row, col = col, out_idx = 85*b +5+ class_num)
            
            flag[yolo_num]=1
            count[yolo_num][row,col,:] += 1
            
            
        for i in range(3):
            for row in range(13*(2**i)):
                for col in range(13*(2**i)):
                    if count[i][row,col,0] == 0:
                        count[i][row,col,:]=1
        
        for i in range(3):
            if (flag[i]!=0):
                gradients[i] /= count[i]
                update_layer[i] = update_layer[i] + l_r * gradients[i]
        
        yolos = seq_infer_model.predict([update_layer[0],update_layer[1],update_layer[2],prev_feature[0],prev_feature[1],prev_feature[2]])
        prev_feature = copy.deepcopy(yolos[:3])
        yolos = yolos[:3]
        print(time.time()-start_time)

        
    
    #yolos = infer_model.predict(process_image)
    boxes = []

    for i in range(len(yolos)):
        # decode the output of the network
        yolo_anchors = anchors[(2 - i) * 6:(3 - i) * 6]
        boxes += decode_netout(yolos[i][0], yolo_anchors, obj_thresh, net_h, net_w, i)
    #print(boxes)
    # correct the sizes of the bounding boxes
    correct_yolo_boxes(boxes, image_h, image_w, net_h, net_w)

    # suppress non-maximal boxes
    do_nms(boxes, nms_thresh)

    # draw bounding boxes on the image using labels
    rows, cols, bs, class_nums, yolo_nums = draw_boxes(img, boxes, labels, obj_thresh, debug = False)
    cv2.imwrite('outputs/new_seq_attend/'+img_name+img_num2str+'_detected.jpg',cv2.resize(img,(1280,720)))
    cv2.imshow('video with bboxes', cv2.resize(img,(1280,720)))
    cv2.waitKey(6)
#     print()

0.10630035400390625
0.10778689384460449
0.1077876091003418
0.10678410530090332
0.10878992080688477
0.10828804969787598
0.11129641532897949
0.11229801177978516
0.10727167129516602
0.1072852611541748
0.10777831077575684
0.10680747032165527
0.10678410530090332
0.11178874969482422
0.10778665542602539
0.10828304290771484
0.10929059982299805
0.10878610610961914
0.10778665542602539
0.10678386688232422
0.10928988456726074
0.10327434539794922
0.10277390480041504
0.10277414321899414
0.10377669334411621
0.10477828979492188
0.1072847843170166
0.10377621650695801
0.10477805137634277
0.10377645492553711
0.10728597640991211
0.10979199409484863
0.10377621650695801
0.10427737236022949
0.12433075904846191
0.1107943058013916
0.10979199409484863
0.10678315162658691
0.10878944396972656
0.10728573799133301
0.10879039764404297
0.10778665542602539
0.1072847843170166
0.10878920555114746
0.10778665542602539
0.10878682136535645
0.1077871322631836
0.10779976844787598
0.10878896713256836
0.10728597640991211
0.1107